In [1]:
import random
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
import math

In [2]:
# In this file, we'll explore some of the ways that p-values can me misinterpreted.  
# Let's do linear regressions for random inputs and random outputs.

covariates = 15
samples = 100
tests = 1000
p_bound = .05
reject_counts = {i:0 for i in range(covariates + 1)}

for i in range(tests):
    X = np.array([[random.random() for i in range(covariates + 1)] for j in range(samples)])
    y = X[:,covariates]
    count = 0
    for j in range(covariates):
        X_constants = sm.add_constant(X[:,j])
        model_sm = sm.OLS(y, X_constants).fit()
        p_values = model_sm.pvalues
        if p_values[1] <= p_bound:
            count += 1

    reject_counts[count] += 1

print(reject_counts)

{0: 450, 1: 358, 2: 154, 3: 30, 4: 6, 5: 2, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0}


The number associated to 0 is the number of times that the regressions correctly reject correlations amongst all variables. For 15 covariates, 100 samples, and 1000 tests, this should happen only about 463 times. So more than half of the time, we will be making false correlations, since we know everything is random! Conclusion: as you increase the number of covariates, you are more likely to make false correlations. The alpha error must be made smaller to compensate for this.

In [3]:
# Computes probabilities assuming that the alpha error is precisely the chance of false correlation for each covariate.
# Reject_counts, covariates, and tests are defined in the previous cell.

probabilities = {}

for i in range(covariates + 1):
    prob = (math.factorial(covariates) / (math.factorial(i) * math.factorial(covariates-i))) * (1-p_bound)**(covariates-i) * (p_bound)**(i)
    probabilities[i] = round(prob, 3)

print(probabilities)

# Let's compare these probabilities to the probabilities we got experimentally.

for i in range(covariates + 1):
    reject_counts[i] = round(reject_counts[i]/tests, 3)

print(reject_counts)

{0: 0.463, 1: 0.366, 2: 0.135, 3: 0.031, 4: 0.005, 5: 0.001, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0}
{0: 0.45, 1: 0.358, 2: 0.154, 3: 0.03, 4: 0.006, 5: 0.002, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0}


Let's see what we can change our bound on p-values to in order to ensure that we maintain an alpha error of less than .05.

In [5]:
alpha = .05

# We solve for the p_bound in the first term of the probability calculation above (the term for not rejecting the null hypothesis).
p_bound = 1 - (1 - alpha)**(1/covariates)

print(p_bound)

0.0034137129465903193


In [6]:
# Redoing the first cell, but with the newly computed p_bound.
covariates = 15
samples = 100
tests = 1000
reject_counts = {i:0 for i in range(covariates + 1)}

for i in range(tests):
    X = np.array([[random.random() for i in range(covariates + 1)] for j in range(samples)])
    y = X[:,covariates]
    count = 0
    for j in range(covariates):
        X_constants = sm.add_constant(X[:,j])
        model_sm = sm.OLS(y, X_constants).fit()
        p_values = model_sm.pvalues
        if p_values[1] <= p_bound:
            count += 1

    reject_counts[count] += 1

print(reject_counts)

{0: 947, 1: 52, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0}


Now the null hypothesis is rejected much less often!